In [ ]:
import pandas as pd
import numpy as np
import sklearn as sl
import seaborn as sns

import math as m

In [ ]:
ds = pd.read_csv('./data/train.csv', index_col='id')
ds.head(3)

In [ ]:
ds_ss = pd.read_csv('./data/sample_submission.csv', index_col='id')
ds_ss.head(3)

In [ ]:
ds_ss.loc[ds_ss['stress'] != 1]

In [ ]:
ds_test = pd.read_csv('./data/test.csv', index_col='id')
ds_test.head(3)

In [ ]:
pd.set_option('display.max_rows', 10)
ds_test.describe()

In [ ]:
ds.describe()

# Поиск шумов

In [ ]:
ds_copy = ds.copy()

In [ ]:
# «а» «у» «о» «и» «э» «ы» «я» «ю» «е» «ё»
vowels = ['а', 'е', 'ё', 'у', 'о', 'и', 'э', 'ы', 'ю', 'я']

#  Анализ данных

In [ ]:
ds[1000:1030]

* можно на выход подавать номер слога
* можно на выход подавать номер ударной буквы, тогда НС будет как бы бинарно классифицировать падает ли на эту букву ударение

# Посчитаем длину слов

In [ ]:
ds['words_lenght'] = [len(word) for word in ds['word']]
ds.sample(5)

In [ ]:
max_words_lenght = np.max(ds['words_lenght'])
max_words_lenght

In [ ]:
ds['lemmas_lenght'] = [len(word) for word in ds['lemma']]
ds.sample(5)

In [ ]:
max_lemms_lenght = np.max(ds['lemmas_lenght'])
max_lemms_lenght

In [ ]:
min_lemms_lenght = np.min(ds['lemmas_lenght'])
min_lemms_lenght

In [ ]:
ds.loc[ds['lemmas_lenght'] <= 2]

In [ ]:
two_letter_words = ['еж', 'мы', 'уж', 'ус', 'яз', 'юг']

In [ ]:
very_short_words = ds.loc[(ds['lemmas_lenght'] <= 2) & (~ds['lemma'].isin(two_letter_words))]

In [ ]:
ds = ds.drop(index=very_short_words.index, axis=1)

In [ ]:
bad_indexes = ds.loc[(ds['num_syllables'] == 1) & (~ds['lemma'].isin(two_letter_words))].index
ds = ds.drop(index=bad_indexes, axis=1)
len(bad_indexes)

In [ ]:
# pd.set_option('display.max_rows', None)
three_letters_words = ds.loc[(ds['lemmas_lenght'] == 3) & (ds['words_lenght'] == 3)]
three_letters_words

In [ ]:
len(ds['word'].unique())

In [ ]:
len(ds['word'])

In [ ]:
ds.describe()

In [ ]:
ds.sample(1)

In [ ]:
sns.histplot(data = {"stress": ds['stress']}, 
             kde=True)

In [ ]:
len(ds.loc[ds['stress'] == 6])

In [ ]:
len(ds.loc[ds['stress'] == 5])

In [ ]:
len(ds.loc[ds['stress'] == 4])

## Выяснение весов классов

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Получение весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(ds['stress']), y=ds['stress'])
class_weights

In [ ]:
ds.loc[ds['stress'] == 2]

## Балансировка даных в ручную

## Unsamplyng

In [ ]:
ds.loc[ds['lemma'] == 'яшка']

In [ ]:
df_empty = ds[0:0]
df_empty

In [ ]:
uniq_lemma_2 = np.unique(ds.loc[ds['stress'] == 2]['lemma'])
len(uniq_lemma_2)

In [ ]:
nds_2 = df_empty

In [ ]:
for uniq_lemma in uniq_lemma_2:
    one_uniq_lemmas_word = ds.loc[(ds['lemma'] == uniq_lemma) & (ds['stress'] == 2)].sample(1)
    nds_2 = pd.concat([nds_2, one_uniq_lemmas_word], ignore_index=True, axis=0)
nds_2.reset_index(drop=True)

In [ ]:
uniq_lemma_3 = np.unique(ds.loc[ds['stress'] == 3]['lemma'])
len(uniq_lemma_3)

In [ ]:
nds_3 = df_empty

In [ ]:
for uniq_lemma in uniq_lemma_3:
    one_uniq_lemmas_word = ds.loc[(ds['lemma'] == uniq_lemma) & (ds['stress'] == 3)].sample(1)
    nds_3 = pd.concat([nds_3, one_uniq_lemmas_word], ignore_index=True, axis=0)
nds_3.reset_index(drop=True)

## Create a 1 class datas

In [ ]:
uniq_lemma_1 = np.unique(ds.loc[ds['stress'] == 1]['lemma'])
len(uniq_lemma_1)

In [ ]:
nds_1 = df_empty

In [ ]:
for uniq_lemma in uniq_lemma_1:
    try:
        one_uniq_lemmas_word = ds.loc[(ds['lemma'] == uniq_lemma) & (ds['stress'] == 1)].sample(2)
    except ValueError:
        one_uniq_lemmas_word = ds.loc[(ds['lemma'] == uniq_lemma) & (ds['stress'] == 1)].sample(1)
    nds_1 = pd.concat([nds_1, one_uniq_lemmas_word], ignore_index=True, axis=0)
nds_1.reset_index(drop=True)

## Create new DS with 1 and new 2 and 3 classes

In [ ]:
nds = pd.concat([nds_1, nds_2, nds_3], ignore_index=True, axis=0)
nds = nds.reset_index(drop=True)

## Oversampling

In [ ]:
words_with_4_s = ds.loc[ds['stress'] == 4].reset_index(drop=True)
for _ in range(0, int(10000/len(words_with_4_s))):
    nds = pd.concat([nds, words_with_4_s], ignore_index=True, axis=0)
    nds.reset_index(drop=True)

In [ ]:
words_with_5_s = ds.loc[ds['stress'] == 5].reset_index(drop=True)

In [ ]:
words_with_6_s = ds.loc[ds['stress'] == 6].reset_index(drop=True)

In [ ]:
nds = pd.concat([nds, words_with_5_s, words_with_6_s], ignore_index=True, axis=0)
nds = nds.reset_index(drop=True)

In [ ]:
words_with_5_s = ds.loc[ds['stress'] == 5].reset_index(drop=True)
for _ in range(0, int(10000/len(words_with_5_s))):
    nds = pd.concat([nds, words_with_5_s], ignore_index=True, axis=0)
    nds.reset_index(drop=True)

In [ ]:
words_with_6_s = ds.loc[ds['stress'] == 6].reset_index(drop=True)
for _ in range(0, int(10000/len(words_with_6_s))):
    nds = pd.concat([nds, words_with_6_s], ignore_index=True, axis=0)
    nds.reset_index(drop=True)

In [ ]:
sns.histplot(data = {"stress": nds['stress']}, 
             kde=True)

In [ ]:
sns.histplot(data = {"words_lenght": nds['words_lenght']}, 
             kde=True)

In [ ]:
nds.loc[nds['words_lenght'] == 13]

### Old experiments

# Токенизируем слово и лемму

In [ ]:
nds = pd.concat([nds, nds], ignore_index=True, axis=0)
ds = nds.reset_index(drop=True)

In [ ]:
letter_list = ['а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я']
letter_by_number = {}
i = 1
for letter in letter_list:
    letter_by_number[letter] = i
    i+=1
letter_by_number

In [ ]:
len(letter_by_number)

In [ ]:
# Tokenize words
for i in range(0, max_words_lenght):
    ds[f'wl{i+1}'] = [(letter_by_number[word[i]] if len(word) > i else 0) for word in ds['word']]
    # is letter vowel?
    # ds[f'is_wl_v{i+1}'] = [(1 if (len(word) > i and word[i] in vowels) else 0) for word in ds['word']]
    # # How many vowels in the word?
    # i = 0
    # num_lemmas_syllables = 0
    # for id_ in ds['id']:
    #     word = ds.loc[ds['id'] == id_]['word']
    #     for letter in word:
    #         if letter in vowels:
    #             num_lemmas_syllables += 1
    #     ds.loc[ds['id'] == id_, 'num_lemmas_syllables'] = num_lemmas_syllables

In [ ]:
# pd.set_option('display.max_columns', None)
ds.sample(3)

In [ ]:
# pd.set_option('display.max_columns', None)
ds.drop(['word', 'lemma'], axis=1).corr()

In [ ]:
ds.drop(['word', 'lemma', 'words_lenght', 'lemmas_lenght'], axis=1).corr()['stress'].to_list()

# Prepare data

In [ ]:
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, LeakyReLU, Conv1D, MaxPooling1D, Embedding, Flatten
import tensorflow as tf
from tensorflow.keras import Sequential
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# ds_ = ds.drop(['id', 'word', 'lemma', 'words_lenght', 'lemmas_lenght'], axis=1)
ds_ = ds.drop(['word', 'lemma'], axis=1)
# ds_ = nds.drop(['word', 'lemma'], axis=1)

In [ ]:
from sklearn.utils import shuffle
ds_ = shuffle(ds_)

In [ ]:
# X, y = df.values[:, :-1], df.values[:, -1]
X = ds_.drop(['stress', 'num_syllables', 'words_lenght', 'lemmas_lenght'], axis=1)
# y = ds_['stress']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')
y = pd.DataFrame(encoder.fit_transform(ds_[['stress']]).toarray())
# y = pd.DataFrame(ds_[['stress']])

In [ ]:
# Получение весов классов
class_weights = compute_class_weight('balanced', classes=np.unique(ds_['stress']), y=ds_['stress'])
class_weights

In [ ]:
i = 0
class_weights_dict = {}
for e in class_weights:
    class_weights_dict[e] = e
    i+=1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, shuffle=True)

In [ ]:
pd.set_option('display.max_columns', None)
X_train

In [ ]:
y_train

In [ ]:
# Нормализуем данные
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# NN_normalizators.append(scaler) 

In [ ]:
n_futures = len(X.columns)
n_futures

# Trying Ansables

# Trying AI

In [ ]:
def get_model():
    # Create a model
    model = Sequential()
    
    # model.add(BatchNormalization())
    
    # Add input layer
    # model.add(Dense(83, activation='softmax', input_shape = (n_futures,) ))
    # model.add(Dense(100, activation='relu', input_shape = (n_futures,) ))
    model.add(Embedding(input_dim=n_futures*2, output_dim=40))
    
    # model.add(Dropout(0.1))
    # model.add(Dense(100, activation='relu'))
    
    model.add(Dropout(0.1))
    # model.add(Dropout(0.25))
    
    model.add(BatchNormalization())
    # model.add(Conv1D(filters=20 , kernel_size=4, strides=1, padding='same', activation='relu'))
    model.add(Conv1D(filters=20, kernel_size=10, strides=1, padding='same', activation='elu'))
    # model.add(MaxPooling1D(2))
    model.add(MaxPooling1D(1))
    
    # model.add(Dense(100, activation='relu', input_shape = (n_futures,) ))
    
    model.add(Dense(200, activation='relu'))
    model.add(Dropout(0.1))
    # model.add(Dense(100, activation='relu'))
    # model.add(Dropout(0.1))
    # model.add(Dense(100, activation='relu'))
    
    # model.add(Conv1D(filters=20 , kernel_size=4, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling1D(2))
    # model.add(Dense(100, activation='relu'))
    
    # model.add(Dropout(0.2))
    
    model.add(BatchNormalization())
    
    model.add(Flatten())
    
    # output layer
    model.add(Dense(6, activation='softmax'))
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  # optimizer='adam', 
                  # optimizer='rmsprop',
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'],
                 )
    return model

In [ ]:
# # # Compile

# # model.compile(loss='mse',              
# #               # optimizer=Adam(learning_rate=0.2), 
# #               # optimizer='adam',
# #               optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
# #               # metrics=['mae'])
# #               metrics=['mse'])

model = get_model()
print(model.summary()) # Выводим структуру сети
# Train NN
history=model.fit(X_train,
                  y_train,
                  batch_size=300,
                  epochs=50,
                  class_weight=class_weights_dict,
                  # Подавляем вывол протоколов обучения.
                  # verbose=0,
                  # Используем 20% обучающих данных для валидации.
                  # validation_split = 0.2)
                 )



In [ ]:
plt.plot(history.history['accuracy'])
# plt.plot(history.history['hindle'])
plt.show()

In [ ]:
X

In [ ]:
from sklearn.model_selection import KFold

kFold = KFold(5, shuffle=True, random_state=1)

for train, test in kFold.split(X, y):
    model = None
    model = get_model()
    # history = model.evaluate(X.iloc[train, :], y.iloc[train, :]) #, X.iloc[test, :], y.iloc[test, :])
    history=model.fit(X.iloc[train, :],
                      y.iloc[train, :],
                      batch_size=300,
                      epochs=50,
                      class_weight=class_weights_dict,
                      # Подавляем вывол протоколов обучения.
                      # verbose=0,
                      # Используем 20% обучающих данных для валидации.
                      # validation_split = 0.2)
                     )
    plt.plot(history.history['accuracy'])
    # plt.plot(history.history['hindle'])
    plt.show()